# 회원 탈퇴를 예측하는 테크닉

#### 이용 데이터 수정

In [3]:
import pandas as pd
customer = pd.read_csv('/Users/icda/pythondata/pyda100-master/5장/customer_join.csv')
uselog_months = pd.read_csv('/Users/icda/pythondata/pyda100-master/5장/use_log_months.csv')

In [5]:
year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"},inplace=True)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]]
    del tmp_before["연월"]
    tmp_before.rename(columns={"count":"count_1"},inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

/var/folders/3d/kdyrst7d5x92jntwy268lvj40000gn/T/ipykernel_5797/2704685537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns={"count":"count_0"},inplace=True)
/var/folders/3d/kdyrst7d5x92jntwy268lvj40000gn/T/ipykernel_5797/2704685537.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.rename(columns={"count":"count_1"},inplace=True)
/var/folders/3d/kdyrst7d5x92jntwy268lvj40000gn/T/ipykernel_5797/2704685537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [6]:
import pandas as pd

year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"] == year_months[i]].copy()  # 복사본 생성
    tmp.rename(columns={"count": "count_0"}, inplace=True)  # 직접 원본 데이터프레임에 작업
    tmp_before = uselog_months.loc[uselog_months["연월"] == year_months[i - 1]].copy()  # 복사본 생성
    del tmp_before["연월"]
    tmp_before.rename(columns={"count": "count_1"}, inplace=True)  # 직접 원본 데이터프레임에 작업
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


#### 탈퇴 전월의 탈퇴 고객 데이터 작성

from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
exit_uselog.head()

In [8]:
from dateutil.relativedelta import relativedelta
import pandas as pd

exit_customer = customer.loc[customer["is_deleted"] == 1].copy()  # 복사본 생성
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
    exit_customer.at[i, "exit_date"] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)

exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
exit_uselog.head()


33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#결측치 제거
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog

6
6


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
5008,201806,OA078433,7,2.0,XXXXX,C03,M,2016-10-01,2018-08-31,CA1,...,6000.0,일반,3.600000,2.0,7.0,2.0,1.0,2018-08-31,22.0,2018-06-30 00:00:00
9072,201807,TS397712,4,2.0,XXXX,C02,F,2016-11-01,2018-09-30,CA1,...,7500.0,일반,3.000000,2.5,5.0,2.0,1.0,2018-09-30,22.0,2018-07-31 00:00:00
12784,201809,AS934534,1,3.0,XXXX,C01,M,2016-10-01,2018-09-30,CA1,...,10500.0,일반,2.833333,3.0,4.0,1.0,1.0,2018-09-30,23.0,2018-09-30 00:00:00
14524,201809,OA422514,3,2.0,XXXXXX,C01,F,2016-10-01,2018-09-30,CA1,...,10500.0,일반,2.666667,2.5,4.0,1.0,0.0,2018-09-30,23.0,2018-09-30 00:00:00
19952,201811,HI529562,1,2.0,XXXXX,C03,M,2016-12-01,2018-11-30,CA1,...,6000.0,일반,2.875000,2.5,6.0,1.0,1.0,2018-11-30,23.0,2018-11-30 00:00:00
23452,201812,IK703077,2,3.0,XXX,C01,M,2017-01-01,2018-12-31,CA1,...,10500.0,일반,4.555556,4.0,7.0,2.0,1.0,2018-12-31,23.0,2018-12-31 00:00:00


#### 지속 회원 데이터 작성

In [11]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [12]:
#데이터 중복 제거
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201901,HI057067,9,7.0,XXXXXX,C01,F,2018-08-01,NaN,CA1,...,종일,10500.0,일반,8.000000,8.0,10.0,7.0,1.0,2019-04-30,8.0
1,201902,HD708558,4,3.0,XXXXX,C01,M,2015-10-01,NaN,CA1,...,종일,10500.0,일반,4.583333,4.0,7.0,3.0,1.0,2019-04-30,42.0
2,201806,HI099779,8,NaN,XXXXX,C01,M,2018-06-04,NaN,CA2,...,종일,10500.0,입회비반액할인,6.700000,7.0,9.0,4.0,1.0,2019-04-30,10.0
3,201811,HD666936,7,8.0,XXXX,C01,M,2017-11-01,NaN,CA1,...,종일,10500.0,일반,6.333333,6.5,11.0,2.0,1.0,2019-04-30,17.0
4,201903,HI486664,8,9.0,XXXXX,C03,F,2018-10-15,NaN,CA1,...,야간,6000.0,일반,8.333333,8.0,9.0,8.0,1.0,2019-04-30,6.0


In [13]:
#지속회원 데이터와 탈퇴회원 데이터 결합
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

2848


/var/folders/3d/kdyrst7d5x92jntwy268lvj40000gn/T/ipykernel_5797/3709746153.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201901,HI057067,9,7.0,XXXXXX,C01,F,2018-08-01,NaT,CA1,...,10500.0,일반,8.000000,8.0,10.0,7.0,1.0,2019-04-30,8.0,NaN
1,201902,HD708558,4,3.0,XXXXX,C01,M,2015-10-01,NaT,CA1,...,10500.0,일반,4.583333,4.0,7.0,3.0,1.0,2019-04-30,42.0,NaN
2,201806,HI099779,8,NaN,XXXXX,C01,M,2018-06-04,NaT,CA2,...,10500.0,입회비반액할인,6.700000,7.0,9.0,4.0,1.0,2019-04-30,10.0,NaN
3,201811,HD666936,7,8.0,XXXX,C01,M,2017-11-01,NaT,CA1,...,10500.0,일반,6.333333,6.5,11.0,2.0,1.0,2019-04-30,17.0,NaN
4,201903,HI486664,8,9.0,XXXXX,C03,F,2018-10-15,NaT,CA1,...,6000.0,일반,8.333333,8.0,9.0,8.0,1.0,2019-04-30,6.0,NaN


#### 예측할 달의 재적 기간 작성

In [14]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201901,HI057067,9,7.0,XXXXXX,C01,F,2018-08-01,NaT,CA1,...,8.000000,8.0,10.0,7.0,1.0,2019-04-30,8.0,NaN,5,2019-01-01
1,201902,HD708558,4,3.0,XXXXX,C01,M,2015-10-01,NaT,CA1,...,4.583333,4.0,7.0,3.0,1.0,2019-04-30,42.0,NaN,40,2019-02-01
2,201806,HI099779,8,NaN,XXXXX,C01,M,2018-06-04,NaT,CA2,...,6.700000,7.0,9.0,4.0,1.0,2019-04-30,10.0,NaN,0,2018-06-01
3,201811,HD666936,7,8.0,XXXX,C01,M,2017-11-01,NaT,CA1,...,6.333333,6.5,11.0,2.0,1.0,2019-04-30,17.0,NaN,12,2018-11-01
4,201903,HI486664,8,9.0,XXXXX,C03,F,2018-10-15,NaT,CA1,...,8.333333,8.0,9.0,8.0,1.0,2019-04-30,6.0,NaN,4,2019-03-01


#### 결측치 제거

In [17]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               200
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [18]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2642
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2642
period                  0
now_date                0
dtype: int64

#### 문자열 변수 처리를 위한 가공

In [22]:
target_col = ["campaign_name","class_name","gender","count_1","routine_flg","period","is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,종일,F,7.0,1.0,5,0.0
1,일반,종일,M,3.0,1.0,40,0.0
3,일반,종일,M,8.0,1.0,12,0.0
4,일반,야간,F,9.0,1.0,4,0.0
5,입회비반액할인,종일,F,5.0,1.0,10,0.0


In [23]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,7.0,1.0,5,0.0,True,False,False,False,True,False,True,False
1,3.0,1.0,40,0.0,True,False,False,False,True,False,False,True
3,8.0,1.0,12,0.0,True,False,False,False,True,False,False,True
4,9.0,1.0,4,0.0,True,False,False,True,False,False,True,False
5,5.0,1.0,10,0.0,False,False,True,False,True,False,True,False


In [24]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,7.0,1.0,5,0.0,False,False,True,False,True
1,3.0,1.0,40,0.0,False,False,True,False,False
3,8.0,1.0,12,0.0,False,False,True,False,False
4,9.0,1.0,4,0.0,False,False,False,False,True
5,5.0,1.0,10,0.0,False,True,True,False,True


#### 탈퇴 예측 모델 구축

In [45]:
#의사결정 트리
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 0.]


In [46]:
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
10,0.0,1.0
1,1.0,1.0
11,0.0,0.0


In [47]:
print(len(predict_data))

2648


#### 예측모델 평가와 튜닝

In [50]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.6666666666666666


In [51]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.6666666666666666
1.0


In [52]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

1.0
1.0


#### 모델에 기여하는 변수 확인

In [53]:
importance = pd.DataFrame({"feature_names": X.columns, "coefficient": model.feature_importances_ })
importance

,feature_names,coefficient
0,count_1,0.375
1,routine_flg,0.000
2,period,0.625
3,campaign_name_입회비무료,0.000
4,campaign_name_입회비반액할인,0.000
5,class_name_종일,0.000
6,class_name_주간,0.000
7,gender_F,0.000


#### 회원 탈퇴 예측

In [54]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [56]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1,0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0,1]
elif campaign_name == "일반":
    campaign_name_list = [0,0]
if class_name == "종일":
    class_name_list = [1,0]
elif class_name == "주간":
    class_name_list = [0,1]
elif class_name == "야간":
    class_name_list = [0,0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [58]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]


/Users/icda/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/icda/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
